# CNN
## Imports

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms


#import numpy as np
#import matplotlib.pyplot as plt

## Network

In [3]:
class Network(nn.Module):
    
    def __init__(self):
        super(Network, self).__init__()
        #The words kernal and filter are interchangeable
        self.conv1=nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5) 
        self.conv2=nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1=nn.Linear(in_features= 12*4*4, out_features=120)
        self.fc2=nn.Linear(in_features= 120, out_features=60)
        self.out=nn.Linear(in_features= 60, out_features=10)
        
    def forward(self, t):
        # (1) input layer
        t = t
        
        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)
        
        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)
        
        # (6) output layer
        t = self.out(t)
        #t = F.softmax(t, dim=1)
        
        return t


## Data

In [4]:
train_set = torchvision.datasets.FashionMNIST(
    root = './data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

## Training

In [5]:
%%time
network = Network()

def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(),lr=0.01)

for epoch in range(10):
    total_loss = 0
    total_correct = 0

    for batch in train_loader:
        images, labels = batch
        preds = network(images)
        loss = F.cross_entropy(preds, labels)

        optimizer.zero_grad()
        loss.backward() # calculates gradient
        optimizer.step() # update weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(f"epoch: {epoch}, total_correc: {total_correct}, loss: {total_loss}")
    
# cpu mac: Wall time: 3min 10s
# cpu home: 
# gpu home:


epoch: 0, total_correc: 47641, loss: 331.71572333574295
epoch: 1, total_correc: 51460, loss: 233.35103204846382
epoch: 2, total_correc: 52117, loss: 212.62347108125687
epoch: 3, total_correc: 52542, loss: 203.52921551465988
epoch: 4, total_correc: 52713, loss: 197.0048868060112
epoch: 5, total_correc: 52850, loss: 194.96347169578075
epoch: 6, total_correc: 52947, loss: 191.549305960536
epoch: 7, total_correc: 52966, loss: 189.81929643452168
epoch: 8, total_correc: 53302, loss: 181.736770555377
epoch: 9, total_correc: 53273, loss: 182.00665912032127
CPU times: user 18min 48s, sys: 45 s, total: 19min 33s
Wall time: 3min 12s


## Training on GPU

In [6]:
%%time
network = Network()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)


def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(),lr=0.01)

for epoch in range(10):
    total_loss = 0
    total_correct = 0

    for batch in train_loader:
        images, labels = batch[0].to(device), batch[1].to(device)
        preds = network(images)
        loss = F.cross_entropy(preds, labels)

        optimizer.zero_grad()
        loss.backward() # calculates gradient
        optimizer.step() # update weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(f"epoch: {epoch}, total_correc: {total_correct}, loss: {total_loss}")

cpu
epoch: 0, total_correc: 47222, loss: 337.46857437491417
epoch: 1, total_correc: 51470, loss: 231.14599557220936
epoch: 2, total_correc: 51887, loss: 216.7275089174509
epoch: 3, total_correc: 52372, loss: 205.03373081982136
epoch: 4, total_correc: 52575, loss: 199.91458359360695
epoch: 5, total_correc: 52567, loss: 198.07638183236122
epoch: 6, total_correc: 52897, loss: 189.37615332007408
epoch: 7, total_correc: 53112, loss: 186.7090909332037
epoch: 8, total_correc: 52994, loss: 187.443355768919
epoch: 9, total_correc: 53277, loss: 181.96383042633533
CPU times: user 18min 41s, sys: 39.9 s, total: 19min 21s
Wall time: 3min 6s


## Prediction

In [35]:
sample = next(iter(train_set))
image, label = sample

output = network(image.unsqueeze(0))
pred = torch.argmax(output, dim=1).item()

print(f"Our pred: {pred} actual label: {label}")

Our pred: 5 actual label: 9
